## Import

In [ ]:
import openai,os,json
from dotenv import load_dotenv, find_dotenv; _ = load_dotenv(find_dotenv())
openai.api_key = os.environ['OPENAI_API_KEY']

## Example from OpenAI Document

In [2]:
def get_current_weather(location, unit="fahrenheit"):
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

def run_conversation(input):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{"role": "user", "content": input}],
        functions=[
            {
                "name": "get_current_weather",
                "description": "指定した場所の現在の天気を取得",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "都市と州",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            }
        ],
        function_call="auto",
    )
    message = response["choices"][0]["message"]
    print("message>>>\n", message, "\n\n")

    if message.get("function_call"):
        function_name = message["function_call"]["name"]

        function_response = get_current_weather(
            location=message.get("location"),
            unit=message.get("unit"),
        )

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[
                {"role": "user", "content": input},
                message,
                {
                    "role": "function",
                    "name": function_name,
                    "content": function_response,
                },
            ],
        )
        return second_response

print("response>>>\n", run_conversation("ボストンの天気はどうですか？")["choices"][0]["message"]["content"], "\n\n")

message>>>
 {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston\"\n}"
  }
} 


response>>>
 ボストンの現在の天気は晴れで、風も強いです。気温は72度です。 




## More Complex Example

In [3]:
recipe_text = """\
シンプルなトマトパスタのレシピです。
材料: パスタ（お好みの種類）: 100g、トマト缶: 60g、にんにく: 1かけ、オリーブオイル: 大さじ1、塩: 適量、こしょう: 適量、パルメザンチーズ（お好みで）: 適量
鍋にお湯を沸かし、パスタを袋の表示通りに茹でます。
別のフライパンにオリーブオイルを熱し、みじん切りにしたにんにくを加えて弱火で炒めます。
トマト缶を加え、塩とこしょうで味を調えます。
トマトソースを弱火で加熱し、少しとろみがつくまで煮込みます。
茹で上がったパスタをソースに加え、全体がよく絡まるように混ぜます。
お皿に盛り付け、お好みでパルメザンチーズをかけて完成です。\
"""

In [4]:
recipe_format = {
    'name': 'string: レシピ名。',
    'appliances': ['string: 使用機器'],
    'ingredients': [
        {'name': 'string: 材料名','amount': 'string: 分量'}
    ],
    'instructions': ['string: 手順']
}

In [5]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": recipe_text}],
    functions=[
        {
            "name": "get_recipe",
            "description": "イベント情報をJson形式で返す",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string", "description": "レシピ名"
                    },
                    "appliances": {
                        "type": "array", "description": "使用機器のリスト",
                        "items": {
                            "type": "string", "description": "使用機器"
                        }
                    },
                    "ingredients": {
                        "type": "object", "description": "材料のリスト",
                        "properties": {
                            "name": {
                                "type": "string", "description": "材料名"
                            },
                            "amount": {
                                "type": "string", "description": "材料の分量。例：200g, 大さじ1, 2個, etc",
                            }
                        }
                    },
                    "instructions": {
                        "type": "array", "description": "調理手順のリスト",
                        "items": {
                            "type": "string", "description": "調理手順"
                        }
                    },
                },
                # "required": ["name","appliances","ingredients","instructions"],
            },
        }
    ],
    function_call={"name": "get_recipe"},
)
message = response["choices"][0]["message"]
print(json.dumps(json.loads(message['function_call']['arguments']),indent=2,ensure_ascii=False))

{
  "name": "シンプルなトマトパスタ",
  "ingredients": [
    {
      "name": "パスタ",
      "amount": "100g"
    },
    {
      "name": "トマト缶",
      "amount": "60g"
    },
    {
      "name": "にんにく",
      "amount": "1かけ"
    },
    {
      "name": "オリーブオイル",
      "amount": "大さじ1"
    },
    {
      "name": "塩",
      "amount": "適量"
    },
    {
      "name": "こしょう",
      "amount": "適量"
    },
    {
      "name": "パルメザンチーズ（お好みで）",
      "amount": "適量"
    }
  ],
  "instructions": [
    "鍋にお湯を沸かし、パスタを袋の表示通りに茹でます。",
    "別のフライパンにオリーブオイルを熱し、みじん切りにしたにんにくを加えて弱火で炒めます。",
    "トマト缶を加え、塩とこしょうで味を調えます。",
    "トマトソースを弱火で加熱し、少しとろみがつくまで煮込みます。",
    "茹で上がったパスタをソースに加え、全体がよく絡まるように混ぜます。",
    "お皿に盛り付け、お好みでパルメザンチーズをかけて完成です。"
  ]
}
